# API REST para Predicción de Abandono Estudiantil

Autores:

Silvana Gutierrez

Yesika Rojas

Angel Gutierrez

Jaime Patiño

Javier Gomez

---

## Objetivo

Desarrollar una API REST utilizando **FastAPI** que permita predecir el riesgo de abandono estudiantil mediante un modelo de Random Forest Classifier.

In [1]:
!pip install fastapi uvicorn scikit-learn joblib pydantic nest_asyncio

## Generación del Dataset y Entrenamiento del Modelo

### Características del Dataset:
- **Tamaño:** 1000 estudiantes simulados
- **Variables predictoras:**
  - edad (18-35 años)
  - horas_estudio_semanal (0-30 horas)
  - asistencia (50%-100%)
  - promedio_notas (0.0-5.0)
  - trabaja (0=No, 1=Sí)
- **Variable objetivo:** abandona (0=No abandona, 1=Abandona)

### Lógica de Etiquetado:
Un estudiante se considera en riesgo de abandono si cumple **al menos una** de estas condiciones:
- Estudia menos de 10 horas semanales
- Tiene un promedio menor a 2.5
- Trabaja Y tiene asistencia menor al 70%

### Modelo:
- **Algoritmo:** Random Forest Classifier
- **N° de árboles:** 100
- **División:** 80% entrenamiento, 20% prueba

In [2]:
#Generar dataset simulado y entrenamiento del modelo

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Reproducibilidad
np.random.seed(42)

# Simulación de datos (1000 estudiantes, 5 variables)
data = pd.DataFrame({
    "edad": np.random.randint(18, 35, 1000),
    "horas_estudio_semanal": np.random.normal(15, 5, 1000).clip(0),
    "asistencia": np.random.uniform(0.5, 1.0, 1000),
    "promedio_notas": np.random.normal(3.0, 0.5, 1000).clip(0, 5),
    "trabaja": np.random.randint(0, 2, 1000)
})

# Regla para generar etiqueta (abandona = 1, no abandona = 0)
data["abandona"] = (
    (data["horas_estudio_semanal"] < 10).astype(int) |
    (data["promedio_notas"] < 2.5).astype(int) |
    ((data["trabaja"] == 1) & (data["asistencia"] < 0.7)).astype(int)
)

# Separar features y target
X = data.drop(columns=["abandona"])
y = data["abandona"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluación
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Guardar modelo
joblib.dump(model, "modelo_abandono.pkl")

Accuracy: 1.0


['modelo_abandono.pkl']

## PASO 3: Implementación de la API con FastAPI

En este paso implementamos la API REST con los siguientes componentes:

### Endpoints definidos:
1. **GET /** - Información general de la API
2. **GET /salud** - Verificación del estado del servicio
3. **GET /estadisticas** - Métricas de uso
4. **POST /predecir** - Realizar predicción de abandono

### Sistema de Logging:
- Registro de todas las solicitudes
- Registro de predicciones exitosas
- Registro de errores
- Almacenamiento en archivo `api_logs.log`

### Sistema de Monitoreo:
- Contador de predicciones totales
- Contador de predicciones exitosas
- Contador de predicciones fallidas
- Tiempo de actividad del servicio

In [3]:
# IMPLEMENTACIÓN DE LA API CON FASTAPI

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import nest_asyncio
import uvicorn
import logging
from datetime import datetime
import threading

# Configurar logging básico
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('api_logs.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Habilitar FastAPI en Colab
nest_asyncio.apply()

# Cargar modelo
model = joblib.load("modelo_abandono.pkl")
logger.info(" Modelo cargado correctamente")

# Variables para monitoreo
stats = {
    "total_predicciones": 0,
    "exitosas": 0,
    "fallidas": 0,
    "inicio": datetime.now()
}

# Crear API
app = FastAPI(
    title="API Predicción Abandono Estudiantil",
    description="API REST para predecir abandono estudiantil",
    version="1.0.0"
)

# Modelo de datos de entrada
class Estudiante(BaseModel):
    edad: int = Field(..., ge=16, le=70, description="Edad del estudiante")
    horas_estudio_semanal: float = Field(..., ge=0, description="Horas de estudio semanales")
    asistencia: float = Field(..., ge=0.0, le=1.0, description="Porcentaje de asistencia")
    promedio_notas: float = Field(..., ge=0.0, le=5.0, description="Promedio académico")
    trabaja: int = Field(..., ge=0, le=1, description="¿Trabaja? 0=No, 1=Sí")

    class Config:
        schema_extra = {
            "example": {
                "edad": 22,
                "horas_estudio_semanal": 12.5,
                "asistencia": 0.85,
                "promedio_notas": 3.5,
                "trabaja": 1
            }
        }

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


## 📍 PASO 4: Definición de Endpoints

### Validaciones con Pydantic:
- **edad:** Entre 16 y 70 años
- **horas_estudio_semanal:** Mayor o igual a 0
- **asistencia:** Entre 0.0 y 1.0
- **promedio_notas:** Entre 0.0 y 5.0
- **trabaja:** 0 (No) o 1 (Sí)

### Clasificación de Riesgo:
- **BAJO:** probabilidad < 0.3
- **MEDIO:** 0.3 ≤ probabilidad < 0.6
- **ALTO:** probabilidad ≥ 0.6

In [4]:
# ENDPOINTS DE LA API


@app.get("/")
def inicio():
    """Endpoint de bienvenida"""
    logger.info("Acceso al endpoint raíz")
    return {
        "mensaje": "API de Predicción de Abandono Estudiantil",
        "version": "1.0.0",
        "endpoints": ["/predecir", "/salud", "/estadisticas", "/docs"]
    }

@app.get("/salud")
def verificar_salud():
    """Verificar estado del servicio"""
    logger.info("Verificación de salud")
    return {
        "estado": "activo",
        "timestamp": datetime.now().isoformat(),
        "modelo": "operativo"
    }

@app.get("/estadisticas")
def obtener_estadisticas():
    """Consultar estadísticas de uso"""
    logger.info("Consulta de estadísticas")
    uptime = (datetime.now() - stats["inicio"]).total_seconds()
    return {
        "total_predicciones": stats["total_predicciones"],
        "exitosas": stats["exitosas"],
        "fallidas": stats["fallidas"],
        "tiempo_activo_segundos": round(uptime, 2)
    }

@app.post("/predecir")
def predecir(estudiante: Estudiante):
    """Realizar predicción de abandono estudiantil"""
    try:
        logger.info(f"Nueva predicción - Datos: {estudiante.dict()}")

        # Preparar datos
        datos = pd.DataFrame([{
            'edad': estudiante.edad,
            'horas_estudio_semanal': estudiante.horas_estudio_semanal,
            'asistencia': estudiante.asistencia,
            'promedio_notas': estudiante.promedio_notas,
            'trabaja': estudiante.trabaja
        }])

        # Predicción
        prediccion = int(model.predict(datos)[0])
        probabilidad = float(model.predict_proba(datos)[0][1])

        # Clasificar riesgo
        if probabilidad < 0.3:
            riesgo = "BAJO"
        elif probabilidad < 0.6:
            riesgo = "MEDIO"
        else:
            riesgo = "ALTO"

        resultado = {
            "abandona": prediccion,
            "probabilidad": round(probabilidad, 2),
            "riesgo": riesgo,
            "timestamp": datetime.now().isoformat()
        }

        # Actualizar estadísticas
        stats["total_predicciones"] += 1
        stats["exitosas"] += 1

        logger.info(f"Predicción exitosa - Resultado: {resultado}")
        return resultado

    except Exception as e:
        stats["total_predicciones"] += 1
        stats["fallidas"] += 1
        logger.error(f"Error en predicción: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Error: {str(e)}")


## Ejecución de la API

La API se ejecuta en:
- **Local:** http://127.0.0.1:8000
- **Documentación:** http://127.0.0.1:8000/docs (Swagger UI automático)

**Nota:** La API se ejecuta en un thread separado para permitir continuar con el notebook.

In [5]:
# EJECUTAR LA API

import time

def run_api():
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="warning")

# Iniciar API en thread
thread = threading.Thread(target=run_api, daemon=True)
thread.start()

# Esperar a que la API esté lista
print(" Iniciando API...")
for i in range(10):
    try:
        import requests
        response = requests.get("http://127.0.0.1:8000/salud", timeout=1)
        if response.status_code == 200:
            print("="*60)
            print("API INICIADA CORRECTAMENTE")
            print("="*60)
            print("URL: http://127.0.0.1:8000")
            print("="*60 + "\n")
            break
    except:
        time.sleep(2)
        print(f"   Intento {i+1}/10...")
else:
    print("La API tardó en iniciar, pero debería estar lista pronto...")

 Iniciando API...
API INICIADA CORRECTAMENTE
URL: http://127.0.0.1:8000



##Exposición Pública con ngrok

Para permitir acceso desde herramientas externas como Postman, exponemos la API públicamente usando ngrok.

**¿Qué hace ngrok?**
- Crea un túnel seguro desde Internet hacia nuestro servidor local
- Genera una URL pública (ej: https://xxxxx.ngrok-free.dev)
- Permite probar la API desde cualquier lugar

**Importante:** La URL de ngrok cambia cada vez que se reinicia.

In [8]:
# Instalar y configurar ngrok
!pip install pyngrok -q

from pyngrok import ngrok
import time

# Esperar a que la API esté lista
time.sleep(2)

# Configuración authtoken
ngrok.set_auth_token("33I5O6Qm7Hk7if9O6ddk1JLQYge_68argjVAEvbDvJvwTK6aY")

# Crear túnel público
public_url = ngrok.connect(8000)

print("="*60)
print("URL PÚBLICA PARA POSTMAN")
print("="*60)
print(f"\n{public_url}\n")
print("Endpoints disponibles:")
print(f"  - GET  {public_url}/salud")
print(f"  - POST {public_url}/predecir")
print(f"  - GET  {public_url}/estadisticas")
print("\nCopia esta URL y úsala en Postman")
print("="*60)

URL PÚBLICA PARA POSTMAN

NgrokTunnel: "https://ambrose-interradial-kayla.ngrok-free.dev" -> "http://localhost:8000"

Endpoints disponibles:
  - GET  NgrokTunnel: "https://ambrose-interradial-kayla.ngrok-free.dev" -> "http://localhost:8000"/salud
  - POST NgrokTunnel: "https://ambrose-interradial-kayla.ngrok-free.dev" -> "http://localhost:8000"/predecir
  - GET  NgrokTunnel: "https://ambrose-interradial-kayla.ngrok-free.dev" -> "http://localhost:8000"/estadisticas

Copia esta URL y úsala en Postman


##Pruebas de la API

A continuación realizamos pruebas automáticas de todos los endpoints:

### Casos de prueba:
1. **Health Check** - Verificar que la API responde
2. **Predicción Bajo Riesgo** - Estudiante con buenos indicadores
3. **Predicción Alto Riesgo** - Estudiante con indicadores críticos
4. **Estadísticas** - Consultar métricas de uso

### Resultados esperados:
- Todas las pruebas deben retornar status 200 OK
- Las predicciones deben ser coherentes con los datos de entrada
- Las estadísticas deben reflejar el uso correcto

In [9]:
# PRUEBAS DE LA API

import requests
import json

print("PRUEBAS DE LA API")

base_url = "http://127.0.0.1:8000"

# Verificar que la API responde
try:
    requests.get(f"{base_url}/salud", timeout=2)
except:
    print(" Esperando a que la API responda...")
    time.sleep(5)

# Prueba 1: Salud
print(" PRUEBA 1: Verificar salud")
r = requests.get(f"{base_url}/salud")
print(f"Status: {r.status_code}")
print(json.dumps(r.json(), indent=2) + "\n")

# Prueba 2: Predicción (Bajo riesgo)
print(" PRUEBA 2: Estudiante bajo riesgo")
estudiante1 = {
    "edad": 20,
    "horas_estudio_semanal": 25,
    "asistencia": 0.95,
    "promedio_notas": 4.2,
    "trabaja": 0
}
r = requests.post(f"{base_url}/predecir", json=estudiante1)
print(f"Status: {r.status_code}")
print(json.dumps(r.json(), indent=2) + "\n")

# Prueba 3: Predicción (Alto riesgo)
print(" PRUEBA 3: Estudiante alto riesgo")
estudiante2 = {
    "edad": 28,
    "horas_estudio_semanal": 5,
    "asistencia": 0.55,
    "promedio_notas": 2.1,
    "trabaja": 1
}
r = requests.post(f"{base_url}/predecir", json=estudiante2)
print(f"Status: {r.status_code}")
print(json.dumps(r.json(), indent=2) + "\n")

# Prueba 4: Estadísticas
print(" PRUEBA 4: Consultar estadísticas")
r = requests.get(f"{base_url}/estadisticas")
print(f"Status: {r.status_code}")
print(json.dumps(r.json(), indent=2) + "\n")

print("PRUEBAS COMPLETADAS")

PRUEBAS DE LA API
 PRUEBA 1: Verificar salud
Status: 200
{
  "estado": "activo",
  "timestamp": "2025-10-02T02:09:00.794114",
  "modelo": "operativo"
}

 PRUEBA 2: Estudiante bajo riesgo
Status: 200
{
  "abandona": 0,
  "probabilidad": 0.01,
  "riesgo": "BAJO",
  "timestamp": "2025-10-02T02:09:00.831182"
}

 PRUEBA 3: Estudiante alto riesgo
Status: 200
{
  "abandona": 1,
  "probabilidad": 1.0,
  "riesgo": "ALTO",
  "timestamp": "2025-10-02T02:09:00.868742"
}

 PRUEBA 4: Consultar estadísticas
Status: 200
{
  "total_predicciones": 4,
  "exitosas": 4,
  "fallidas": 0,
  "tiempo_activo_segundos": 996.78
}

PRUEBAS COMPLETADAS


/tmp/ipython-input-1644020794.py:40: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  logger.info(f"Nueva predicción - Datos: {estudiante.dict()}")
/tmp/ipython-input-1644020794.py:40: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  logger.info(f"Nueva predicción - Datos: {estudiante.dict()}")


## Resultados Obtenidos

### Modelo de Machine Learning:
- **Accuracy:** 100% en conjunto de prueba
- **Modelo guardado:** modelo_abandono.pkl

### API REST:
- **4 endpoints** implementados y funcionales
- **Sistema de logging** operativo
- **Sistema de monitoreo** funcionando
- **Validación de datos** con Pydantic

### Pruebas:
- **Health check:** Exitoso
- **Predicciones:** Funcionando correctamente
- **Estadísticas:** Reportando métricas
- **Validación de errores:** Detectando datos inválidos

---

## Pruebas con Postman

La API fue probada exhaustivamente con Postman:

### Casos probados:
1. Estudiante de bajo riesgo
2. Estudiante de alto riesgo
3. Validación de datos incorrectos
4. Consulta de estadísticas

---

## Conclusiones

1. **FastAPI** facilita enormemente la creación de APIs REST con validación automática
2. **Pydantic** proporciona validación robusta de datos de entrada
3. El sistema de **logging** permite rastrear el uso y detectar problemas
4. **ngrok** es útil para exposición rápida sin necesidad de servidor
5. El modelo Random Forest alcanzó **100% de accuracy** con reglas claras

---

## Enlace repositorio

- **Repositorio GitHub:** https://github.com/SilvanaGCh/API_Predicci-n_Abandono_Estudiantil/tree/main
